In [8]:
import pandas as pd
import sys
import csv
from statistics import median

In [9]:
results = pd.read_csv("final_results_for_plotting.csv")

In [10]:
results['project'] = results['project'].str.replace('.csv','')

/var/folders/09/2vzp5xpd2ldd4ggk4xzjt1qr0000gn/T/ipykernel_77681/2064200378.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  results['project'] = results['project'].str.replace('.csv','')


In [11]:
results

,Unnamed: 0,Unnamed: 0.1,project,algorithm,batch_size,builds_reqd,median_delay,testall_size,sum_delay,method,confidence
0,0,0,rubinius,BATCH4,4,30.751498,5.0,2169,2960.0,sbs,NaN
1,1,1,rubinius,BATCHBISECT,1,30.751498,5.0,2169,2960.0,sbs,NaN
2,2,2,rubinius,BATCHBISECT,2,30.751498,5.0,2169,2960.0,sbs,NaN
3,3,3,rubinius,BATCHBISECT,4,30.751498,5.0,2169,2960.0,sbs,NaN
4,4,4,rubinius,BATCHBISECT,8,30.751498,5.0,2169,2960.0,sbs,NaN
...,...,...,...,...,...,...,...,...,...,...,...
615,135,135,geoserver,BATCH4,4,85.496185,2.0,1179,NaN,tr,16.0
616,144,144,rubinius,BATCH4,4,69.847855,2.0,2169,NaN,tr,20.0
617,153,153,cloudify,BATCH4,4,34.025375,2.0,2601,NaN,tr,19.0
618,162,162,cloud_controller_ng,BATCH4,4,38.811729,2.0,1296,NaN,tr,20.0


In [12]:
results = results.drop('Unnamed: 0', axis='columns')

In [13]:
method = ['mlci', 'tr', 'gr', '']
algorithms = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']
batchsize = [2, 4, 8, 16]
project_list = list(set(results['project'].tolist()))

In [14]:
for m in method:
    
    filename = m + '_pivot_median_delay.csv'
    file = open(filename, 'w')
    res_headers = ['project', 'B4_builds', 'B4_delay', 'B4_Batch', 'BB_builds', 'BB_delay', 'BB_Batch', 'BS4_builds', 'BS4_delay', 'BS4_Batch']
    res_writer = csv.writer(file)
    res_writer.writerow(res_headers)
    
    
    for p in project_list:
        
        b4_builds = 0
        b4_delay = 0
        b4_batch = 0
        
        bs4_builds = 0
        bs4_delay = 0
        bs4_batch = 0
        
        bb_builds = 0
        bb_delay = 0
        bb_batch = 0
        
        for alg in algorithms:
            data = results[ (results['method'] == m) & (results['algorithm'] == alg) & (results['project'] == p)]
            data = data.drop(data[data.batch_size < 2].index)
            data = data.drop(data[data.batch_size > 16].index)

            best_batch = 0
            lowest_distance = sys.maxsize
            delays = data['median_delay'].tolist()
            builds_reqd = data['builds_reqd'].tolist()
            batch_size = data['batch_size'].tolist()

            for i in range(len(delays)):
                d = (builds_reqd[i]**2 + delays[i]**2)**0.5
                if d < lowest_distance:
                    
                    best_batch = batch_size[i]
                    lowest_distance = d
                    
                    if alg == 'BATCHBISECT':
                        bb_builds = builds_reqd[i]
                        bb_delay = delays[i]
                        bb_batch = best_batch
                    elif alg == 'BATCHSTOP4':
                        bs4_builds = builds_reqd[i]
                        bs4_delay = delays[i]
                        bs4_batch = best_batch
                    else:
                        b4_builds = builds_reqd[i]
                        b4_delay = delays[i]
                        b4_batch = best_batch
            print('{} - {} - {} - {}'.format(p, m, alg, best_batch))
        res_writer.writerow([p, b4_builds, b4_delay, b4_batch, bb_builds, bb_delay, bb_batch, bs4_builds, bs4_delay, bs4_batch])
        

metasploit-framework - mlci - BATCH4 - 4
metasploit-framework - mlci - BATCHBISECT - 16
metasploit-framework - mlci - BATCHSTOP4 - 16
rails - mlci - BATCH4 - 4
rails - mlci - BATCHBISECT - 2
rails - mlci - BATCHSTOP4 - 8
fog - mlci - BATCH4 - 4
fog - mlci - BATCHBISECT - 2
fog - mlci - BATCHSTOP4 - 8
heroku - mlci - BATCH4 - 4
heroku - mlci - BATCHBISECT - 2
heroku - mlci - BATCHSTOP4 - 8
geoserver - mlci - BATCH4 - 4
geoserver - mlci - BATCHBISECT - 2
geoserver - mlci - BATCHSTOP4 - 8
graylog2-server - mlci - BATCH4 - 4
graylog2-server - mlci - BATCHBISECT - 16
graylog2-server - mlci - BATCHSTOP4 - 16
loomio - mlci - BATCH4 - 4
loomio - mlci - BATCHBISECT - 2
loomio - mlci - BATCHSTOP4 - 8
opal - mlci - BATCH4 - 4
opal - mlci - BATCHBISECT - 4
opal - mlci - BATCHSTOP4 - 8
gradle - mlci - BATCH4 - 4
gradle - mlci - BATCHBISECT - 4
gradle - mlci - BATCHSTOP4 - 8
rubinius - mlci - BATCH4 - 4
rubinius - mlci - BATCHBISECT - 2
rubinius - mlci - BATCHSTOP4 - 8
puppet - mlci - BATCH4 - 4
pup

In [15]:
batch_reps = {0: 'na', 2: '2', 4: '4', 8:'8', 16:'16'}

In [16]:
print(batch_reps[2])

2


In [17]:
ssr = pd.read_csv('tr_pivot_median_delay.csv')

In [18]:
ssr = ssr.sort_values(by=['project'])
projects = ssr['project'].tolist()
col3 = ssr['B4_builds'].tolist()
col4 = ssr['B4_delay'].tolist()
b4_batch = ssr['B4_Batch'].tolist()
col5 = ssr['BB_builds'].tolist()
col6 = ssr['BB_delay'].tolist()
bb_batch = ssr['BB_Batch'].tolist()
col7 = ssr['BS4_builds'].tolist()
col8 = ssr['BS4_delay'].tolist()
bs4_batch = ssr['BS4_Batch'].tolist()

In [19]:
for i in range(len(projects)):
    print("{} & {} & {} & {} & {} & {} & {} & {} & {} & {}\\\\".format(projects[i].split('.')[0],  
                                                                round(col3[i], 3), round(col4[i], 3), batch_reps[b4_batch[i]], 
                                                                round(col5[i], 3), round(col6[i], 3), batch_reps[bb_batch[i]], 
                                                                round(col7[i], 3), round(col8[i], 3), batch_reps[bs4_batch[i]] ))
    print("\hline")

print("Median & {} & {} & & {} & {} & & {} & {}".format(round(median(col3), 3), round(median(col4), 3), round(median(col5), 3), round(median(col6), 3), round(median(col7), 3), round(median(col8), 3)))
print("\hline\n\hline")

cloud_controller_ng & 38.812 & 2.0 & 4 & 30.247 & 1.0 & 2 & 48.457 & 4.0 & 8\\
\hline
cloudify & 34.025 & 2.0 & 4 & 33.064 & 1.0 & 2 & 39.216 & 4.0 & 8\\
\hline
concerto & 16.667 & 2.0 & 4 & 12.399 & 1.0 & 2 & 20.692 & 4.0 & 8\\
\hline
fog & 56.217 & 2.0 & 4 & 49.008 & 1.0 & 2 & 65.542 & 4.0 & 8\\
\hline
geoserver & 85.496 & 2.0 & 4 & 94.232 & 1.0 & 2 & 98.388 & 4.0 & 8\\
\hline
gradle & 8.879 & 2.0 & 4 & 7.373 & 1.0 & 2 & 9.761 & 4.0 & 8\\
\hline
graylog2-server & 9.099 & 2.0 & 4 & 7.437 & 1.0 & 2 & 9.449 & 4.0 & 8\\
\hline
heroku & 35.911 & 2.0 & 4 & 28.356 & 1.0 & 2 & 43.022 & 4.0 & 8\\
\hline
jruby & 110.664 & 2.0 & 4 & 118.779 & 1.0 & 2 & 123.429 & 4.0 & 8\\
\hline
loomio & 42.677 & 2.0 & 4 & 34.596 & 1.0 & 2 & 51.641 & 4.0 & 8\\
\hline
metasploit-framework & 9.064 & 2.0 & 4 & 7.334 & 1.0 & 2 & 10.38 & 4.0 & 8\\
\hline
opal & 15.527 & 2.0 & 4 & 11.823 & 1.0 & 2 & 19.088 & 4.0 & 8\\
\hline
open-build-service & 43.049 & 2.0 & 4 & 34.479 & 1.0 & 2 & 54.061 & 4.0 & 8\\
\hline
orbeon-f

In [20]:
ml = pd.read_csv('mlci_pivot_median_delay.csv')
ml = ml.sort_values(by='project')

In [21]:
projects = ml['project'].tolist()
col3 = ml['B4_builds'].tolist()
col4 = ml['B4_delay'].tolist()
b4_batch = ml['B4_Batch'].tolist()
col5 = ml['BB_builds'].tolist()
col6 = ml['BB_delay'].tolist()
bb_batch = ml['BB_Batch'].tolist()
col7 = ml['BS4_builds'].tolist()
col8 = ml['BS4_delay'].tolist()
bs4_batch = ml['BS4_Batch'].tolist()

In [22]:
for i in range(len(projects)):
    print("{} & {} & {} & {} & {} & {} & {} & {} & {} & {}\\\\".format(projects[i].split('.')[0], 
                                                                round(col3[i], 2), int(round(col4[i], 3)), batch_reps[b4_batch[i]], 
                                                                round(col5[i], 2), int(round(col6[i], 3)), batch_reps[bb_batch[i]], 
                                                                round(col7[i], 2), int(round(col8[i], 3)), batch_reps[bs4_batch[i]] ))
    print("\hline")
print("Median & {} & {} & & {} & {} & & {} & {}".format(round(median(col3), 3), round(median(col4), 3), round(median(col5), 3), round(median(col6), 3), round(median(col7), 3), round(median(col8), 3)))
print("\hline\n\hline")

cloud_controller_ng & 62.27 & 2 & 4 & 55.56 & 1 & 2 & 74.61 & 4 & 8\\
\hline
cloudify & 21.11 & 2 & 4 & 23.95 & 2 & 4 & 28.49 & 4 & 8\\
\hline
concerto & 39.45 & 2 & 4 & 40.42 & 4 & 8 & 38.49 & 4 & 8\\
\hline
fog & 69.78 & 2 & 4 & 64.09 & 1 & 2 & 86.31 & 4 & 8\\
\hline
geoserver & 87.02 & 2 & 4 & 71.25 & 1 & 2 & 103.39 & 4 & 8\\
\hline
gradle & 37.59 & 2 & 4 & 38.63 & 2 & 4 & 37.85 & 4 & 8\\
\hline
graylog2-server & 30.71 & 2 & 4 & 23.01 & 8 & 16 & 21.96 & 8 & 16\\
\hline
heroku & 36.27 & 2 & 4 & 38.13 & 1 & 2 & 42.31 & 4 & 8\\
\hline
jruby & 73.06 & 2 & 4 & 62.62 & 1 & 2 & 103.6 & 4 & 8\\
\hline
loomio & 54.1 & 2 & 4 & 57.26 & 1 & 2 & 73.86 & 4 & 8\\
\hline
metasploit-framework & 34.04 & 2 & 4 & 28.53 & 8 & 16 & 28.0 & 8 & 16\\
\hline
opal & 35.4 & 2 & 4 & 36.68 & 2 & 4 & 36.54 & 4 & 8\\
\hline
open-build-service & 51.97 & 2 & 4 & 45.89 & 1 & 2 & 60.49 & 4 & 8\\
\hline
orbeon-forms & 63.33 & 2 & 4 & 67.26 & 1 & 2 & 73.93 & 4 & 8\\
\hline
puppet & 31.21 & 2 & 4 & 23.48 & 4 & 8 & 23.01 

In [23]:
gr = pd.read_csv('gr_pivot_median_delay.csv')
gr = gr.sort_values(by='project')

In [24]:
sbs_values = results[ (results['method'] == 'sbs') & ((results['algorithm'] == 'BATCHSTOP4') & (results['batch_size'] == 8))]

In [25]:
sbs_values = sbs_values.sort_values(by='project')

In [26]:
sbs_values

,Unnamed: 0.1,project,algorithm,batch_size,builds_reqd,median_delay,testall_size,sum_delay,method,confidence
94,106,cloud_controller_ng,BATCHSTOP4,8,11.265432,7.0,1296,3389.0,sbs,NaN
78,88,cloudify,BATCHSTOP4,8,16.955017,16.0,2601,2967.0,sbs,NaN
142,160,concerto,BATCHSTOP4,8,3.864734,30.0,1242,3214.0,sbs,NaN
102,115,fog,BATCHSTOP4,8,26.587302,7.0,1512,3247.0,sbs,NaN
38,43,geoserver,BATCHSTOP4,8,62.001696,4.0,1179,675.0,sbs,NaN
118,133,gradle,BATCHSTOP4,8,0.882658,53.0,1926,8005.0,sbs,NaN
46,52,graylog2-server,BATCHSTOP4,8,1.224847,12.0,1143,1008.0,sbs,NaN
126,142,heroku,BATCHSTOP4,8,14.577778,11.0,1125,1158.0,sbs,NaN
62,70,jruby,BATCHSTOP4,8,79.320367,3.0,4473,1889.0,sbs,NaN
110,124,loomio,BATCHSTOP4,8,17.297980,10.0,1584,6309.0,sbs,NaN


In [27]:
gr['SBS_builds'] = sbs_values['builds_reqd'].tolist()
gr['SBS_delay'] = sbs_values['median_delay'].tolist()
gr.to_csv('baseline_pivot_median_delay.csv')

In [28]:
gr

,project,B4_builds,B4_delay,B4_Batch,BB_builds,BB_delay,BB_Batch,BS4_builds,BS4_delay,BS4_Batch,SBS_builds,SBS_delay
14,cloud_controller_ng,74.691358,1.5,4,80.401235,1.5,4,80.092593,3.5,8,11.265432,7.0
19,cloudify,45.251826,2.0,4,42.868128,8.0,16,38.638985,8.0,16,16.955017,16.0
12,concerto,45.974235,2.0,4,44.122383,4.0,8,42.190016,4.0,8,3.864734,30.0
2,fog,83.730159,1.5,4,93.915344,0.5,2,89.748677,3.5,8,26.587302,7.0
4,geoserver,99.491094,2.0,4,108.227311,1.0,2,108.481764,4.0,8,62.001696,4.0
8,gradle,44.548287,2.0,4,40.965732,4.0,8,39.511942,4.0,8,0.882658,53.0
5,graylog2-server,34.995626,2.0,4,23.797025,8.0,16,22.747157,8.0,16,1.224847,12.0
3,heroku,53.333333,2.0,4,55.644444,4.0,8,51.377778,4.0,8,14.577778,11.0
15,jruby,108.405992,2.0,4,121.372681,1.0,2,119.271183,4.0,8,79.320367,3.0
6,loomio,73.484848,1.5,4,82.828283,1.5,4,78.030303,3.5,8,17.297980,10.0


In [29]:
projects = gr['project'].tolist()
col2 = sbs_values['builds_reqd'].tolist()
col3 = sbs_values['median_delay'].tolist()
col4 = gr['B4_builds'].tolist()
col5 = gr['B4_Batch'].tolist()
col6 = gr['BB_builds'].tolist()
col7 = gr['BB_Batch'].tolist()
col8 = gr['BS4_builds'].tolist()
col9 = gr['BS4_Batch'].tolist()

In [30]:
for i in range(len(projects)):
    print("{} & {} & {} & {} & {} & {} & {} \\\\".format(projects[i], 
                                                                round(col2[i], 2), round(col4[i], 2), 
                                                                round(col6[i], 2), round(col7[i], 2), 
                                                                round(col8[i], 2), round(col9[i], 2) ))
    print("\hline")
print("Median & {} & {} & {} & {} & {} & {}".format(round(median(col2), 3), round(median(col4), 3), round(median(col6), 3), round(median(col7), 3), round(median(col8), 3), round(median(col9), 3)))
print("\hline\n\hline")

cloud_controller_ng & 11.27 & 74.69 & 80.4 & 4 & 80.09 & 8 \\
\hline
cloudify & 16.96 & 45.25 & 42.87 & 16 & 38.64 & 16 \\
\hline
concerto & 3.86 & 45.97 & 44.12 & 8 & 42.19 & 8 \\
\hline
fog & 26.59 & 83.73 & 93.92 & 2 & 89.75 & 8 \\
\hline
geoserver & 62.0 & 99.49 & 108.23 & 2 & 108.48 & 8 \\
\hline
gradle & 0.88 & 44.55 & 40.97 & 8 & 39.51 & 8 \\
\hline
graylog2-server & 1.22 & 35.0 & 23.8 & 16 & 22.75 & 16 \\
\hline
heroku & 14.58 & 53.33 & 55.64 & 8 & 51.38 & 8 \\
\hline
jruby & 79.32 & 108.41 & 121.37 & 2 & 119.27 & 8 \\
\hline
loomio & 17.3 & 73.48 & 82.83 & 4 & 78.03 & 8 \\
\hline
metasploit-framework & 1.02 & 37.48 & 28.97 & 16 & 28.44 & 16 \\
\hline
opal & 3.49 & 43.52 & 40.88 & 8 & 39.46 & 8 \\
\hline
open-build-service & 16.54 & 69.36 & 76.33 & 4 & 72.35 & 8 \\
\hline
orbeon-forms & 64.44 & 80.09 & 96.84 & 2 & 82.82 & 8 \\
\hline
puppet & 1.73 & 35.3 & 25.14 & 16 & 24.51 & 16 \\
\hline
rails & 24.11 & 72.8 & 82.97 & 4 & 76.76 & 8 \\
\hline
rubinius & 30.75 & 84.88 & 92.07 &

In [31]:
ml = pd.read_csv('mlci_pivot_median_delay.csv')
tr = pd.read_csv('tr_pivot_median_delay.csv')
bs = pd.read_csv('baseline_pivot_median_delay.csv')

ml = ml.sort_values(by=['project'])
tr = tr.sort_values(by=['project'])
bs = bs.sort_values(by=['project'])

In [32]:
dfs = [ml, tr]
ms = ['ml', 'tr']
nc = []
for i in range(len(dfs)):
    df = dfs[i]
    columns = df.columns
    nc.append({})
    for c in columns:
        new_name = ms[i] + '_' + c
        nc[i][c] = new_name
    

In [33]:
ml = ml.rename(columns=nc[0])
tr = tr.rename(columns=nc[1])


In [34]:
df = pd.concat([ml, tr, bs], axis=1)

In [35]:
df.columns

Index(['ml_project', 'ml_B4_builds', 'ml_B4_delay', 'ml_B4_Batch',
       'ml_BB_builds', 'ml_BB_delay', 'ml_BB_Batch', 'ml_BS4_builds',
       'ml_BS4_delay', 'ml_BS4_Batch', 'tr_project', 'tr_B4_builds',
       'tr_B4_delay', 'tr_B4_Batch', 'tr_BB_builds', 'tr_BB_delay',
       'tr_BB_Batch', 'tr_BS4_builds', 'tr_BS4_delay', 'tr_BS4_Batch',
       'Unnamed: 0', 'project', 'B4_builds', 'B4_delay', 'B4_Batch',
       'BB_builds', 'BB_delay', 'BB_Batch', 'BS4_builds', 'BS4_delay',
       'BS4_Batch', 'SBS_builds', 'SBS_delay'],
      dtype='object')

In [36]:
df.to_csv('all_pivots.csv')